# Create hierarchy
Creates a hierachy from the wikidata entries by its parent attribute.
The created tree is filtered and saved to json for visualization.
It is also saved back to csv so the reduced number of keywords can be used for clustering.

Uses the csv `keyword_wikidata_small.csv` from the reducing notebook. Could also be used with all keywords.
Expect csv `keyword_wikidata_small.csv` with columns keyword, wikidata and parent0.

In [1]:
#config

#path where csv files are stored and results are saved to
data_path = '../../../data'

In [2]:
import os
import pandas as pd
import copy
import sys
import requests
import json

In [3]:
# load csv file

keyword_wiki_path = os.path.join(data_path, 'keyword_wikidata_small.csv')
df_keywords_wiki = pd.read_csv(keyword_wiki_path, index_col='wikidata')

In [4]:
# remove numerical keywords

df_keywords_wiki = df_keywords_wiki[~df_keywords_wiki.keyword.str.fullmatch('[0-9]+')]

In [5]:
df_keywords_wiki.count()

keyword    60529
parent0    55706
dtype: int64

In [6]:
# get parents to list
def getParents(s):
    s = s[2:-2].split('", "')
    return s

In [7]:
# convert wikidata entries to nodes

nodes = []

for index, row in df_keywords_wiki.iterrows():
    node = {'id': index, 'name': row['keyword'], 'children': []}
    
    if not pd.isnull(row['parent0']):
        parent = getParents(row['parent0'])
        
        node['parent'] = parent
        
    
    # remove cycles
    if node['id'] == 'Q12136':
        parent.remove('Q179630')
        parent.remove('Q814207')
    
    if node['id'] == 'Q954007':
        parent.remove('Q954007')
    
    if node['id'] == 'Q35535':
        parent.remove('Q35535')
        
    if node['id'] == 'Q832237':
        parent.remove('Q20113959')
            
    nodes.append(node)

In [8]:
# add nodes that are not in the data

def addNode(id, name):
    nodes.append({'id': id, 'name': name, 'children': []})
    

addNode('Q17339814', 'chemical substances')
addNode('Q47154513', 'structural class of chemical compounds')
addNode('Q214609', 'base material')

In [9]:
# build tree from the nodes, creates multiple trees

def build_tree(nodes):
    
    nodes = copy.deepcopy(nodes)
    
    node_dict = {}
    
    for node in nodes:
        node_dict[node['id']] = node
        
    parents = node_dict.copy()
    
    notin = 0
        
    for node in nodes:
        if 'parent' in node:
            for parent in node['parent']:
                if parent in node_dict:
                    if node['id'] in parents:
                        del parents[node['id']]
                    node_dict[parent]['children'].append(node)
            
    return list(parents.values())


trees = build_tree(nodes)

In [10]:
# use only trees that have more than one node

trees_with_children = list(filter(lambda x: len(x['children']) > 0, trees))

In [11]:
# number of trees
len(trees_with_children)

1008

In [12]:
def tree_size(node):
    size = 1
    if 'children' not in node:
        return size
    for child in node['children']:
        size += tree_size(child)
    return size

In [13]:
# check for cycles in a combined tree

def find_cycle(node):
    discovered = set()
    finished = set()
    
    for child in node['children']:
        if child['id'] not in discovered and child['id'] not in finished:
            discovered, finished = dfs_visit(child, discovered, finished)
    

def dfs_visit(node, discovered, finished):
    discovered.add(node['id'])

    for child in node['children']:
        # Detect cycles
        if child['id'] in discovered:
            print(f"Cycle detected: found a back edge from {child['name']}, {child['id']} to {node['name']},{node['id']}.")
            
            # remove cycle TODO
            #node['children'].remove(child)

        # Recurse into DFS tree
        if child['id'] not in finished:
            dfs_visit(child, discovered, finished)

    if node['id'] in discovered:
        discovered.remove(node['id'])
    finished.add(node['id'])

    return discovered, finished


dfs_parents = copy.deepcopy(trees_with_children)
dfs_root = {'name': 'root', 'keyword': 'root', 'children': dfs_parents}
find_cycle(dfs_root)


In [14]:
# number of nodes

sizes = [(x['name'], tree_size(x)) for x in trees_with_children]
sum([x[1] for x in sizes])

44014

In [15]:
# use only larger trees
large_nodes = list(filter(lambda x: tree_size(x) > 20, trees_with_children))
# number of trees
len(large_nodes)

96

In [16]:
# number of nodes when only using larger trees
sum(tree_size(x) for x in large_nodes)

40368

In [17]:
# create one tree from all trees
root = {'name': 'root', 'id': 'root', 'children': large_nodes}

In [18]:
root_to_filter = copy.deepcopy(root)

In [19]:
# functions to filter the tree

def remove_subtree(node, id):
    for child in node['children']:
        if child['id'] == id:
            node['children'].remove(child)
        else:
            remove_subtree(child, id)
            
def remove_if_not_in(node, id, parent):
    if node['id'] == id:
        return
    if node['id'] != parent:
        for child in node['children']:
            if child['id'] == id:
                node['children'].remove(child)
    for child in node['children']:
        remove_if_not_in(child, id, parent)

In [20]:
# remove subtrees that seem not necessary for a chemical context

to_remove = [
    'Q3305213', #painting
    'Q11424', #film
    'Q1348305', #erratum
    'Q134556', #single
    'Q93184', #drawing
    'Q871232', #editorial
    'Q125191', #photograph
    'Q41176', #building
    'Q4022', #river
    'Q860861', #sculpture
    'Q8502', #mountain
    'Q20026787', #!chemical component
    'Q4026292', #?action
    'Q24229398', #?agent
    'Q151885', #concept
    'Q71421787', #hydroxides
    'Q61788060', #human activity
    'Q386724', #work
    'Q16889133', #class
    'Q34749', #social science
    'Q21198', #computer science
    'Q7748', #law
    'Q7991', #natural science
    'Q6256', #country
    'Q23442', #island
    'Q34770', #language
    'Q267628', #column
    'Q107425', #landscape
    'Q1076968', #digital media
    'Q1186952', #interactive media
    'Q11024', #communication
    'Q11028', #information
    'Q16387', #information science
    'Q79782', #motion
    'Q131257', #intellectual property
    'Q170658', #creativity
    'Q2684591', #statement
    'Q28797', #applied science
    'Q362482', #operation
    'Q395', #mathematics
    'Q41217', #mechanics
    'Q702492', #urban area
    'Q8008', #earth sciences
    'Q9081', #knowledge
    'Q214339', #role
    'Q3769299', #human behaviour
    'Q11348', #function
]


for r in to_remove:
    remove_subtree(root_to_filter, r)


In [21]:
# remove subtrees that have multiple occurrences in the tree

remove_not_in = [
    ('Q8054', 'Q17339814'), #protein
    ('Q3249551', 'Q1190554'), #process
    ('Q1183543', 'Q39546'), #device
    ('Q39546', 'Q16798631'), #tool
    ('Q42889', 'Q11019'), #vehicle
    ('Q1799072', 'Q2695280'), #method
    ('Q46857', 'Q1799072'), #scientific method
    ('Q1379672', 'Q1799072'), #evaluation
    ('Q133500', 'Q2200417'), #learning
    ('Q156', 'Q174211'), #alcohol
    ('Q217602', 'Q46857'), #analysis
]

for r in remove_not_in:
    remove_if_not_in(root_to_filter, r[0], r[1])

In [22]:
# final tree size

tree_size(root_to_filter)

16299

In [23]:
# remove parents from the visualization tree for a smaller file size

vis_tree = copy.deepcopy(root_to_filter)

def remove_parents(node):
    node.pop('parent', None)
    for child in node['children']:
        remove_parents(child)

remove_parents(vis_tree)

In [24]:
# output tree to json file

out_file = os.path.join(data_path, 'hierarchy_tree.json')

with open(out_file, 'w+') as outfile:
    json.dump(vis_tree, outfile)

In [25]:
# get set of tree nodes

node_set = set()

def traverse_tree_to_set(root):
    global node_set
    if root['id'] != 'root': 
        if 'parent' in root:          
            parent0 = '"{}"'.format(', '.join(root['parent']))
        else:
            parent0 = ''
        node_set.add((root['id'], root['name'], parent0))
    for child in root['children']:
        traverse_tree_to_set(child)
        
traverse_tree_to_set(root_to_filter)

In [26]:
# convert nodes to dataframe

results_keywords = pd.DataFrame(columns=['keyword', 'wikidata', 'parent0'])

for node in node_set:
    if node[2] != '':
        results_keywords = results_keywords.append({'keyword': node[1], 'wikidata': node[0], 'parent0': node[2]}, ignore_index=True)
    else:
        results_keywords = results_keywords.append({'keyword': node[1], 'wikidata': node[0]}, ignore_index=True)

In [27]:
# save filterd keywords to csv

keyword_result_path = os.path.join(data_path, 'keyword_result.csv')
results_keywords.to_csv(keyword_result_path, index = False)

## Further methods for analysis

In [28]:
# calculate sizes of all subtrees

sub_tree_sizes = pd.DataFrame(columns=['keyword', 'id', 'size'])

def tree_size_to_df(node):
    global sub_tree_sizes
    size = 1
    if 'children' not in node:
        sub_tree_sizes = sub_tree_sizes.append({'keyword': node['name'], 'id': node['id'], 'size': size}, ignore_index=True)
        return size
    for child in node['children']:
        size += tree_size_to_df(child)
    sub_tree_sizes = sub_tree_sizes.append({'keyword': node['name'], 'id': node['id'], 'size': size}, ignore_index=True)
    return size

tree_size_to_df(root_to_filter)

16299

In [29]:
# check for larger subtrees that occure multiple times in the tree

grouped_sizes = sub_tree_sizes.groupby(['id', 'keyword']).agg(['count', 'sum'])
grouped_sizes = grouped_sizes.sort_values([('size', 'count')], ascending=False)

grouped_sizes[(grouped_sizes[('size', 'count')] > 1) & (grouped_sizes[('size', 'sum')] > 50)]

size     
                                           count  sum
id        keyword                                    
Q80294    cellulose                           23   69
Q41534    starch                              14   70
Q173600   disaccharide                        12   72
Q407553   glycosaminoglycan                    9   90
Q320607   oligosaccharide                      8   80
Q134219   polysaccharides                      8  512
Q2250497  unsaturated fatty acids              6   60
Q61476    fatty acid                           5   90
Q416796   glucan                               5   90
Q222174   flavonoid                            5  165
Q8066     amino acid                           5   95
Q41581    alkane                               4   68
Q134856   carboxylic acid                      4  244
Q416997   peptide hormones                     4   60
Q422649   biopolymer                           4  404
Q407212   polycyclic aromatic hydrocarbons     4   60
Q123619   nucleic acid                         4   68
Q177911   steroid                              3  111
Q39875001 measure                              3   60
Q43648    hydrocarbon                          3   69
Q679692   biopharmaceutical                    3   87
Q33525    carcinoma                            3  120
Q11364    hormone                              3  102
Q421948   organic acid                         3  303
Q11358    carbohydrate                         3  303
Q516751   polyketide                           3  138
Q283      water                                2   54
Q67008917 biological products                  2   60
Q1979154  model                                2   94
Q3314483  fruit                                2   84
Q131656   pesticide                            2  166
Q11004    vegetable                            2   60
Q1075     color                                2  120
Q967847   nanomaterial                         2   52
Q12078    cancer                               2   88
Q12718609 statistical method                   2   54
Q7189713  physiological condition              2   82
Q1789452  criterion                            2   58
Q170744   ketone                               2   98
Q181322   insecticide                          2   78

# Create keywords - class table

In [30]:
# load all keywords

keywords_path = os.path.join(data_path, 'keyword.csv')
df_keywords = pd.read_csv(keywords_path, index_col='name')

In [31]:
categories = {
    'Q206229': 'biomolecule',
    'Q17339814': 'chemical substance',
    'Q11426': 'metal',
    'Q3249551': 'process',
    'Q4751159': 'analytical method',
    #'': 'biochemical relation',       what id?
    'Q937228': 'property',
    
    # added categories 
    'Q47154513': 'structural class of chemical compounds',
    'disease': 'Q12136',
}

In [32]:
def label_tree(node):
    if node['id'] in categories:
        category = categories[node['id']]
        keyword_category_list.append({'keyword': node['name'], 'category': category})
        for child in node['children']:
            label_subtree(child, category)
    for child in node['children']:
        label_tree(child)
    

def label_subtree(node, category):
    keyword_category_list.append({'keyword': node['name'], 'category': category})
    for child in node['children']:
        label_subtree(child, category)

In [33]:
keyword_category_list = []

label_tree(root_to_filter)

In [34]:
keyword_category_df = pd.DataFrame(keyword_category_list, columns = ['keyword', 'category'])
keyword_category_df = keyword_category_df.drop_duplicates()

In [35]:
other_keywords = df_keywords[~ df_keywords.index.isin(keyword_category_df.keyword)].index.to_list()
others_df = pd.DataFrame({'keyword': other_keywords, 'category': ['Others'] * len(other_keywords)})

In [36]:
keyword_category_result_df = keyword_category_df.append(others_df, ignore_index = True)

In [37]:
keyword_category_result_df.groupby('category').count()

,keyword
category,
Others,37474
analytical method,27
biomolecule,119
chemical substance,2523
metal,36
process,809
property,407
structural class of chemical compounds,1026


In [38]:
# save keyword-class table to csv

keyword_category_result_path = os.path.join(data_path, 'keyword_categories.csv')
keyword_category_result_df.to_csv(keyword_category_result_path, index = False)